In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sys
import datetime, os
from sklearn import metrics
from sklearn import preprocessing

import pandas as pd

column_names = ['sgs_p','rho','tempe','press','rhoe',
                'gradrho', 'gradt', 'gradp','gradroe','laprho','laprhoe','inv1','inv2','inv3']
raw_dataset = pd.read_csv('data.dat', names=column_names,
                      sep=" ",nrows = 10000)

data = raw_dataset.copy()

data.columns = ['sgs_p','rho','tempe','press','rhoe',
                'gradrho', 'gradt', 'gradp','gradroe','laprho','laprhoe','inv1','inv2','inv3']

x = data.values #returns a numpy array
#scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x)

                          
data = pd.DataFrame(x_scaled)
data.columns = ['sgs_p','rho','tempe','press','rhoe',
                'gradrho', 'gradt', 'gradp','gradroe','laprho','laprhoe','inv1','inv2','inv3']

train_dataset = data.sample(frac=0.75,random_state=0)
test_dataset = data.drop(train_dataset.index)

train_labels = train_dataset.pop('sgs_p')
test_labels = test_dataset.pop('sgs_p')

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)
train_dataset = np.asarray(train_dataset)
test_dataset = np.asarray(test_dataset)


def build_model():
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.layers import LeakyReLU

    model = keras.Sequential([
    layers.Dense(10, use_bias=True,activation='relu', input_shape=[train_dataset.shape[1]]),    
    layers.Dense(10, use_bias=True,activation='relu'),     
    layers.Dense(1, use_bias=True,activation='linear'),
    ])

    return model

model = build_model()
model.summary()

def train_model():
  
    
  optimizer = tf.optimizers.Adam() #

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse'])

# The patience parameter is the amount of epochs to check for improvement
  early_stop = keras.callbacks.EarlyStopping(monitor='val_mse',min_delta=1e-17, patience=10)
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1,profile_batch=0)


  model.fit(train_dataset, 
            train_labels, 
            epochs=100,
            batch_size=1000,
            validation_split = 0.5, 
            verbose=0, 
            callbacks=[early_stop,tensorboard_callback])




NameError: name 'preprocessing' is not defined